<a href="https://colab.research.google.com/github/teamgaon/lg_farm/blob/main/20220201_hj_plant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ttach

In [2]:
pip install efficientnet_pytorch

## 할 일

plant-doc dataset으로 pretrain

대회 데이터로 finetune

b2, b4 비교

In [3]:
!nvidia-smi
# k80 -> T4 -> P100

Tue Feb  1 06:26:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 사용 패키지

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import ttach as tta
import albumentations as A
from albumentations.pytorch import ToTensor
from sklearn.model_selection import KFold
import torch.nn as nn

from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b0'  # b6

model = EfficientNet.from_pretrained(model_name, num_classes=7)

Loaded pretrained weights for efficientnet-b0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/LG/plant/train.zip

Archive:  /content/drive/MyDrive/LG/plant/train.zip
replace train/00009_jpg.rf.b4187ecff7dbde2ac358f15e01adcecb.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/drive/MyDrive/LG/plant/test.zip

In [8]:
!unzip /content/drive/MyDrive/LG/PlantDoc-Dataset-master.zip

Archive:  /content/drive/MyDrive/LG/PlantDoc-Dataset-master.zip
5467f6012d78d1c446145d5f582da6096f852ae8
   creating: PlantDoc-Dataset-master/
  inflating: PlantDoc-Dataset-master/LICENSE.txt  
  inflating: PlantDoc-Dataset-master/PlantDoc_Examples.png  
  inflating: PlantDoc-Dataset-master/README.md  
   creating: PlantDoc-Dataset-master/test/
   creating: PlantDoc-Dataset-master/test/Apple Scab Leaf/
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/052609%20Hartman%20Crabapple%20scab%20single%20leaf.JPG.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/1b321015-6e33-4f18-aade-888f4383fe92.jpeg.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/28-500x375.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/816.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/apple%20scab%20leaf.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/apple%20scabnew.jpg  
  inflating: PlantDoc-Dataset-master/test/Apple Scab Leaf/app

## CustomDataset 제작

In [9]:
train_classes = pd.read_table('/content/drive/MyDrive/LG/plant/train_classes.txt',names=['filename', 'Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf','Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Soybean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot'],sep=',')
test_classes = pd.read_table('/content/drive/MyDrive/LG/plant/test_classes.txt',names=['filename', 'Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf','Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Soybean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot'],sep=',')

display(train_classes)
display(test_classes)

,filename,Apple Scab Leaf,Apple leaf,Apple rust leaf,Bell_pepper leaf,Bell_pepper leaf spot,Blueberry leaf,Cherry leaf,Corn Gray leaf spot,Corn leaf blight,Corn rust leaf,Peach leaf,Potato leaf,Potato leaf early blight,Potato leaf late blight,Raspberry leaf,Soyabean leaf,Soybean leaf,Squash Powdery mildew leaf,Strawberry leaf,Tomato Early blight leaf,Tomato Septoria leaf spot,Tomato leaf,Tomato leaf bacterial spot,Tomato leaf late blight,Tomato leaf mosaic virus,Tomato leaf yellow virus,Tomato mold leaf,Tomato two spotted spider mites leaf,grape leaf,grape leaf black rot
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,blueberry-leaf-isolated-white-background-blueb...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2324,role-of-whitefly-in-plant-virus-transmission-b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


,filename,Apple Scab Leaf,Apple leaf,Apple rust leaf,Bell_pepper leaf,Bell_pepper leaf spot,Blueberry leaf,Cherry leaf,Corn Gray leaf spot,Corn leaf blight,Corn rust leaf,Peach leaf,Potato leaf,Potato leaf early blight,Potato leaf late blight,Raspberry leaf,Soyabean leaf,Soybean leaf,Squash Powdery mildew leaf,Strawberry leaf,Tomato Early blight leaf,Tomato Septoria leaf spot,Tomato leaf,Tomato leaf bacterial spot,Tomato leaf late blight,Tomato leaf mosaic virus,Tomato leaf yellow virus,Tomato mold leaf,Tomato two spotted spider mites leaf,grape leaf,grape leaf black rot
0,glyphosate_jpg.rf.01dea2d24a1e3591855a68e077bb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,6134794031202304_jpeg_jpg.rf.021531229ad8d669e...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,squashsilver2_JPG_jpg.rf.02bdb1c8a5c8bdf6f689f...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,peach-leaf-16690421_jpg.rf.030bd0c31f731f86e54...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20180511_091133-24l1vhg-e1526047988236_jpg.rf....,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,toma_TMV_jpg.rf.fadfed6938fbeef46c39f78a02da3b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
235,17fc47_jpg.rf.fe1c3b512f4182d08f00ac2c612ddb1c...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
236,7190f27ead55c0f6e3ff8b982972810a5446a9713a49d_...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
237,636368370164094909-late-blight-15-_jpg.rf.febe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [10]:
def name_to_label(name:str):
  for col in train_classes.columns:
    if train_classes.loc[train_classes['filename'] == name, col].item() == 1:
      return col
  
train_classes['name'] = train_classes['filename'].map(name_to_label)
train_disease = train_classes[['filename','name']]

In [11]:
def name_to_label(name:str):
  for col in test_classes.columns:
    if test_classes.loc[test_classes['filename'] == name, col].item() == 1:
      return col

test_classes['name'] = test_classes['filename'].map(name_to_label)

test_disease = test_classes[['filename','name']]

In [12]:
train_disease

,filename,name
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,Tomato leaf late blight
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,Corn leaf blight
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,Tomato mold leaf
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,Corn leaf blight
4,blueberry-leaf-isolated-white-background-blueb...,Blueberry leaf
...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,Blueberry leaf
2324,role-of-whitefly-in-plant-virus-transmission-b...,Tomato leaf yellow virus
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,grape leaf
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,Tomato leaf bacterial spot


In [13]:
my_dict={}
for i in range(len(train_disease['name'].unique())):
  my_dict[train_disease['name'].unique()[i]] = i

my_dict

{'Apple leaf': 22,
 'Bell_pepper leaf': 10,
 'Bell_pepper leaf spot': 9,
 'Blueberry leaf': 3,
 'Cherry leaf': 24,
 'Corn Gray leaf spot': 21,
 'Corn leaf blight': 1,
 'Corn rust leaf': 5,
 None: 4,
 'Peach leaf': 18,
 'Potato leaf': 27,
 'Potato leaf early blight': 11,
 'Potato leaf late blight': 16,
 'Raspberry leaf': 13,
 'Soyabean leaf': 20,
 'Soybean leaf': 19,
 'Squash Powdery mildew leaf': 15,
 'Strawberry leaf': 23,
 'Tomato Early blight leaf': 17,
 'Tomato Septoria leaf spot': 7,
 'Tomato leaf': 25,
 'Tomato leaf bacterial spot': 12,
 'Tomato leaf late blight': 0,
 'Tomato leaf mosaic virus': 14,
 'Tomato leaf yellow virus': 6,
 'Tomato mold leaf': 2,
 'grape leaf': 26,
 'grape leaf black rot': 8}

In [14]:
def name_to_label(name:str):
  return my_dict[name]

train_disease['name'] = train_disease['name'].map(name_to_label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
train_disease['filename'] = train_disease['filename'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
train_disease

,filename,name
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,0
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,1
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,2
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,1
4,blueberry-leaf-isolated-white-background-blueb...,3
...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,3
2324,role-of-whitefly-in-plant-virus-transmission-b...,6
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,26
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,12


In [28]:
plantdoc = glob('/content/PlantDoc-Dataset-master/train/*/*')
plantdoc

['/content/PlantDoc-Dataset-master/train/Tomato two spotted spider mites leaf/comparing-diseases-4-canker-tomato-1y51ejs.jpg',
 '/content/PlantDoc-Dataset-master/train/Tomato two spotted spider mites leaf/SpotSpeckBlightMite-1l4v879.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/stock-photo-strawberry-leaves-collection-isolated-on-white-163409792.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/stock-photo-strawberry-leaf-isolated-on-white-background-with-clipping-path-568145485.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/Strawberry+leaves.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/strawberry+leaf+eila+collage+obsession.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/1788724-spring-leaves-of-garden-strawberry-macro-background-isolated.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/7.jpg',
 '/content/PlantDoc-Dataset-master/train/Strawberry leaf/IMG_4056.JPG.jpg',
 '/content/PlantDoc-Data

In [71]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train'):
        self.mode = mode
        self.files = files

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        
        # image
        image_path = plantdoc[i]
        img = cv2.imread(image_path)
        img = cv2.resize(img, dsize=(416, 416), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/255
        img = np.transpose(img, (2,0,1))

        if self.mode == 'train':
            # json_path = f'{file}/{file_name}.json'
            # with open(json_path, 'r') as f:
            #     json_file = json.load(f)
            
            # crop = json_file['annotations']['crop']
            # disease = json_file['annotations']['disease']
            # risk = json_file['annotations']['risk']
            
            self.label = train_disease['name'].iloc[i]

            # augmentation = random.randint(0,2)
            # if augmentation==1:
            #     img = img[::-1].copy()
            # elif augmentation==2:
            #     img = img[:,::-1].copy()
            # img = transforms.ToTensor()(img)
            
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                # 'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                'label' : torch.tensor(self.label, dtype=torch.long)
            }
        else:
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                # 'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32)
            }

In [32]:
image_path = plantdoc[0]
img = cv2.imread(image_path)
img = cv2.resize(img, dsize=(416, 416), interpolation=cv2.INTER_AREA)
img = img.astype(np.float32)/255
img = np.transpose(img, (2,0,1))

# 하이퍼파라미터 및 변수

In [31]:
device = torch.device("cuda:0")
batch_size = 16
class_n = len(my_dict)
learning_rate = 1e-4
embedding_dim = 512
# num_features = len(csv_feature_dict)
max_len = 24*6
dropout_rate = 0.001
epochs = 15
k_folds = 5
vision_pretrain = True
save_path = 'best_model.pt'

In [35]:
img

array([[[0.18431373, 0.17254902, 0.15294118, ..., 0.6784314 ,
         0.9411765 , 0.99607843],
        [0.16862746, 0.15294118, 0.13725491, ..., 0.40784314,
         0.9137255 , 0.99215686],
        [0.15686275, 0.14117648, 0.12156863, ..., 0.4745098 ,
         0.9098039 , 0.9843137 ],
        ...,
        [0.25882354, 0.23529412, 0.22352941, ..., 0.2784314 ,
         0.9372549 , 0.99607843],
        [0.25490198, 0.23137255, 0.21960784, ..., 0.27450982,
         0.9372549 , 0.99607843],
        [0.25882354, 0.2627451 , 0.29803923, ..., 0.27058825,
         0.92941177, 0.99215686]],

       [[0.16862746, 0.17254902, 0.18039216, ..., 0.7647059 ,
         0.96862745, 0.9882353 ],
        [0.17254902, 0.1764706 , 0.18039216, ..., 0.50980395,
         0.9529412 , 0.99607843],
        [0.18039216, 0.18431373, 0.18039216, ..., 0.5882353 ,
         0.9647059 , 0.99607843],
        ...,
        [0.32941177, 0.3137255 , 0.31764707, ..., 0.4862745 ,
         1.        , 0.99215686],
        [0.3

In [36]:
torch.tensor(img, dtype=torch.float32)

tensor([[[0.1843, 0.1725, 0.1529,  ..., 0.6784, 0.9412, 0.9961],
         [0.1686, 0.1529, 0.1373,  ..., 0.4078, 0.9137, 0.9922],
         [0.1569, 0.1412, 0.1216,  ..., 0.4745, 0.9098, 0.9843],
         ...,
         [0.2588, 0.2353, 0.2235,  ..., 0.2784, 0.9373, 0.9961],
         [0.2549, 0.2314, 0.2196,  ..., 0.2745, 0.9373, 0.9961],
         [0.2588, 0.2627, 0.2980,  ..., 0.2706, 0.9294, 0.9922]],

        [[0.1686, 0.1725, 0.1804,  ..., 0.7647, 0.9686, 0.9882],
         [0.1725, 0.1765, 0.1804,  ..., 0.5098, 0.9529, 0.9961],
         [0.1804, 0.1843, 0.1804,  ..., 0.5882, 0.9647, 0.9961],
         ...,
         [0.3294, 0.3137, 0.3176,  ..., 0.4863, 1.0000, 0.9922],
         [0.3216, 0.3098, 0.3137,  ..., 0.4784, 1.0000, 0.9922],
         [0.3255, 0.3451, 0.3882,  ..., 0.4745, 1.0000, 0.9922]],

        [[0.2431, 0.2353, 0.2353,  ..., 0.7647, 0.9686, 0.9804],
         [0.2431, 0.2353, 0.2353,  ..., 0.5059, 0.9529, 0.9922],
         [0.2471, 0.2392, 0.2314,  ..., 0.5804, 0.9608, 0.

# 데이터셋 구성

In [40]:
train = sorted(glob('/content/PlantDoc-Dataset-master/train/*'))
test = sorted(glob('/content/PlantDoc-Dataset-master/test/*'))

labelsss = train_disease['name']
# train, val = train_test_split(train, test_size=0.2, stratify=labelsss)

In [41]:
train

['/content/PlantDoc-Dataset-master/train/Apple Scab Leaf',
 '/content/PlantDoc-Dataset-master/train/Apple leaf',
 '/content/PlantDoc-Dataset-master/train/Apple rust leaf',
 '/content/PlantDoc-Dataset-master/train/Bell_pepper leaf',
 '/content/PlantDoc-Dataset-master/train/Bell_pepper leaf spot',
 '/content/PlantDoc-Dataset-master/train/Blueberry leaf',
 '/content/PlantDoc-Dataset-master/train/Cherry leaf',
 '/content/PlantDoc-Dataset-master/train/Corn Gray leaf spot',
 '/content/PlantDoc-Dataset-master/train/Corn leaf blight',
 '/content/PlantDoc-Dataset-master/train/Corn rust leaf',
 '/content/PlantDoc-Dataset-master/train/Peach leaf',
 '/content/PlantDoc-Dataset-master/train/Potato leaf early blight',
 '/content/PlantDoc-Dataset-master/train/Potato leaf late blight',
 '/content/PlantDoc-Dataset-master/train/Raspberry leaf',
 '/content/PlantDoc-Dataset-master/train/Soyabean leaf',
 '/content/PlantDoc-Dataset-master/train/Squash Powdery mildew leaf',
 '/content/PlantDoc-Dataset-master/

In [42]:
train_transforms = A.Compose([
A.Normalize(),
A.HorizontalFlip(),
A.VerticalFlip(p=0.5),
ToTensor()
])

In [43]:
train_dataset = CustomDataset(train)
# val_dataset = CustomDataset(val)
test_dataset = CustomDataset(test, mode = 'test')

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=2, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=False)

# 모델

## 이미지 분류 모델 : efficientnet_b7 -> b4 -> b2 -> b4

In [15]:
class CNN_Encoder(nn.Module):
    def __init__(self, class_n, rate=0.1):
        super(CNN_Encoder, self).__init__()
        self.model = models.efficientnet_b2(pretrained=True,progress=True)

    def forward(self, inputs):
        output = self.model(inputs)
        return output

## 시계열 모델 : LSTM

In [46]:
class RNN_Decoder(nn.Module):
    def __init__(self, max_len, embedding_dim, class_n, rate):
        super(RNN_Decoder, self).__init__()
        self.lstm = nn.LSTM(max_len, embedding_dim)
        self.rnn_fc = nn.Linear(num_features*embedding_dim, 1000)
        self.final_layer = nn.Linear(1000 + 1000, class_n) # resnet out_dim + lstm out_dim
        self.dropout = nn.Dropout(rate)

    def forward(self, enc_out, dec_inp):
        hidden, _ = self.lstm(dec_inp)
        hidden = hidden.view(hidden.size(0), -1)
        hidden = self.rnn_fc(hidden)
        concat = torch.cat([enc_out, hidden], dim=1) # enc_out + hidden 
        fc_input = concat
        output = self.dropout((self.final_layer(fc_input)))
        return output

## 앙상블

In [18]:
class CNN2RNN(nn.Module):
    def __init__(self, max_len, embedding_dim, class_n, rate):
        super(CNN2RNN, self).__init__()
        self.cnn = CNN_Encoder(embedding_dim, rate)
        self.rnn = RNN_Decoder(max_len, embedding_dim, class_n, rate)
        
    def forward(self, img, seq):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq)
        
        return output

In [20]:
model = models.efficientnet_b2(pretrained=True)
model = model.to(device)

NameError: ignored

# 학습

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

criterion = nn.CrossEntropyLoss()

NameError: ignored

In [17]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def train_step(batch_item, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

In [65]:
import random
import torchvision.transforms as transforms
import gc

In [66]:
gc.collect()

396

In [1]:
# loss_plot, val_loss_plot = [], []
# metric_plot, val_metric_plot = [], []
      
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)
  
# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_dataset)):
  gc.collect()
  loss_plot, val_loss_plot = [], []
  metric_plot, val_metric_plot = [], []
  
  # Print
  print('')
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  # Define data loaders for training and testing data in this fold
  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=batch_size, sampler=train_subsampler, num_workers=2)
  val_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=batch_size, sampler=test_subsampler, num_workers=2)
  
  model = models.efficientnet_b6(max_len=max_len, embedding_dim=embedding_dim, class_n=class_n, rate=dropout_rate)
  model = model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                          lr_lambda=lambda epoch: 0.95 ** epoch,
                                          last_epoch=-1,
                                          verbose=False)

  criterion = nn.CrossEntropyLoss()
  
  for epoch in range(epochs):
      total_loss, total_val_loss = 0, 0
      total_acc, total_val_acc = 0, 0
      
      tqdm_dataset = tqdm(enumerate(train_dataloader))
      training = True
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_loss += batch_loss
          total_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
              'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
          })
      loss_plot.append(total_loss/(batch+1))
      metric_plot.append(total_acc/(batch+1))
      
      tqdm_dataset = tqdm(enumerate(val_dataloader))
      training = False
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_val_loss += batch_loss
          total_val_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Val Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
              'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
          })
      val_loss_plot.append(total_val_loss/(batch+1))
      val_metric_plot.append(total_val_acc/(batch+1))
      scheduler.step()

      if np.max(val_metric_plot) == val_metric_plot[-1]:
          torch.save(model.state_dict(), save_path+'best_model'+str(fold)+'.pt')
          print('best')

      

NameError: ignored

## 적정 에포크 8회인듯?

학습은 한 40분 추론은 한 20분? 걸리는듯

# 학습 결과

In [ ]:
# plt.figure(figsize=(10,7))
# plt.grid()
# plt.plot(loss_plot, label='train_loss')
# plt.plot(val_loss_plot, label='val_loss')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.title("Loss", fontsize=25)
# plt.legend()
# plt.show()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.grid()
# plt.plot(metric_plot, label='train_metric')
# plt.plot(val_metric_plot, label='val_metric')
# plt.xlabel('epoch')
# plt.ylabel('metric')
# plt.title("F-1", fontsize=25)
# plt.legend()
# plt.show()

# 추론

In [ ]:
# transforms = tta.Compose(
#     [
#         tta.HorizontalFlip(),
#         tta.VerticalFlip(),
#         tta.Rotate90(angles=[0, 180]),
#         # tta.Scale(scales=[1, 2, 4]),
#         # tta.Multiply(factors=[0.9, 1, 1.1]),     
#     ]
# )

In [ ]:
def predict(dataset, model0, model1, model2, model3, model4):
    model0.eval()
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        seq = batch_item['csv_feature'].to(device)
        with torch.no_grad():
            output = 0.2*model0(img, seq) + 0.2*model1(img, seq) + 0.2*model2(img, seq) + 0.2*model3(img, seq) + 0.2*model4(img, seq)
        output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

model0 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model0.load_state_dict(torch.load(save_path+'best_model'+str(0)+'.pt', map_location=device))
model0.to(device)

model1 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model1.load_state_dict(torch.load(save_path+'best_model'+str(1)+'.pt', map_location=device))
model1.to(device)

model2 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model2.load_state_dict(torch.load(save_path+'best_model'+str(2)+'.pt', map_location=device))
model2.to(device)

model3 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model3.load_state_dict(torch.load(save_path+'best_model'+str(3)+'.pt', map_location=device))
model3.to(device)

model4 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model4.load_state_dict(torch.load(save_path+'best_model'+str(4)+'.pt', map_location=device))
model4.to(device)

preds = predict(test_dataloader, model0, model1, model2, model3, model4)

In [ ]:
preds = np.array([label_decoder[int(val)] for val in preds])

# 제출 파일 생성

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/LG/sample_submission.csv')
submission['label'] = preds
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/LG/submission.csv', index=False)

## 할일
파프리카 흰가루병인 식물의 데이터로 초기, 중기, 말기를 학습하는 모델 생성 -> model2

model1이 파프리카 흰가루병이라고 예측하면 해당 index의 testdata를 model2에 투입하고 결과를 반환

3_a9_1